.pivot() is useful for rearranging tables, and .pivot_table() can do some of the same things as a .groupby() method call

Probably .groupby() is the best approach, but it depends on the circumstance

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Sales_Funnel_CRM.csv')

In [4]:
df.head()

,Account Number,Company,Contact,Account Manager,Product,Licenses,Sale Price,Status
0,2123398,Google,Larry Pager,Edward Thorp,Analytics,150,2100000,Presented
1,2123398,Google,Larry Pager,Edward Thorp,Prediction,150,700000,Presented
2,2123398,Google,Larry Pager,Edward Thorp,Tracking,300,350000,Under Review
3,2192650,BOBO,Larry Pager,Edward Thorp,Analytics,150,2450000,Lost
4,420496,IKEA,Elon Tusk,Edward Thorp,Analytics,300,4550000,Won


Aparently this is a very common type of table to see: they're called CRM (data funnels) and they're used to track the status of accounts/aquisition

In [6]:
#help(pd.pivot)

In [27]:
df['Company'].unique()

array([' Google', 'BOBO', 'IKEA', 'Tesla Inc.', 'Microsoft', 'Walmart',
       'Apple', 'Exxon Mobile', 'ATT', 'CVS Health', 'Salesforce',
       'Cisco'], dtype=object)

In [54]:
df[df['Company'] == ' Google']

,Account Number,Company,Contact,Account Manager,Product,Licenses,Sale Price,Status
0,2123398,Google,Larry Pager,Edward Thorp,Analytics,150,2100000,Presented
1,2123398,Google,Larry Pager,Edward Thorp,Prediction,150,700000,Presented
2,2123398,Google,Larry Pager,Edward Thorp,Tracking,300,350000,Under Review


In [63]:
df['Company'] = df['Company'].str.strip()

In [64]:
df['Company'].unique()

array(['Google', 'BOBO', 'IKEA', 'Tesla Inc.', 'Microsoft', 'Walmart',
       'Apple', 'Exxon Mobile', 'ATT', 'CVS Health', 'Salesforce',
       'Cisco'], dtype=object)

In [66]:
df.head()

,Account Number,Company,Contact,Account Manager,Product,Licenses,Sale Price,Status
0,2123398,Google,Larry Pager,Edward Thorp,Analytics,150,2100000,Presented
1,2123398,Google,Larry Pager,Edward Thorp,Prediction,150,700000,Presented
2,2123398,Google,Larry Pager,Edward Thorp,Tracking,300,350000,Under Review
3,2192650,BOBO,Larry Pager,Edward Thorp,Analytics,150,2450000,Lost
4,420496,IKEA,Elon Tusk,Edward Thorp,Analytics,300,4550000,Won


Ok, I was able to figure out how to strip the excesss space from ' Google', turning it into 'Google'

It's not possible to just do this as df.str.strip() because the df contains ints. I had to actually target the right column by doing df['Company'].str.strip()

In [71]:
licenses = df[['Company', 'Product', 'Licenses']]

In [72]:
licenses

,Company,Product,Licenses
0,Google,Analytics,150
1,Google,Prediction,150
2,Google,Tracking,300
3,BOBO,Analytics,150
4,IKEA,Analytics,300
5,Tesla Inc.,Analytics,300
6,Tesla Inc.,Prediction,150
7,Microsoft,Tracking,300
8,Walmart,Analytics,150
9,Apple,Analytics,300


Now we have only a few columns selected, so it's time to pivot:

In [77]:
pd.pivot(data = licenses, index = 'Company', columns = 'Product', values = 'Licenses')

Product,Analytics,GPS Positioning,Prediction,Tracking
Company,,,,
ATT,NaN,NaN,150.0,150.0
Apple,300.0,NaN,NaN,NaN
BOBO,150.0,NaN,NaN,NaN
CVS Health,NaN,NaN,NaN,450.0
Cisco,300.0,300.0,NaN,NaN
Exxon Mobile,150.0,NaN,NaN,NaN
Google,150.0,NaN,150.0,300.0
IKEA,300.0,NaN,NaN,NaN
Microsoft,NaN,NaN,NaN,300.0


There are many NaN cells because not every product has been sold to every company, i.e. GPS has not been sold to Google

Note: the call is pd.pivot, not df.pivot

If an agregation is to be done on top of this, you can use pivot_table()

In [84]:
pd.pivot_table(df, index = 'Company', aggfunc = 'sum')

,Licenses,Sale Price
Company,,
ATT,300,1050000
Apple,300,4550000
BOBO,150,2450000
CVS Health,450,490000
Cisco,600,4900000
Exxon Mobile,150,2100000
Google,600,3150000
IKEA,300,4550000
Microsoft,300,350000


This is pretty much the same as using the groupby() method

The arguments are (dataframe name, the index used to group, and aggregation method)

Since in this case, the account number doesn't make sense as a column to be summed, I will drop it:

In [85]:
pd.pivot_table(df, index = 'Company', aggfunc = 'sum').drop(columns = 'Account Number', axis = 1)

,Licenses,Sale Price
Company,,
ATT,300,1050000
Apple,300,4550000
BOBO,150,2450000
CVS Health,450,490000
Cisco,600,4900000
Exxon Mobile,150,2100000
Google,600,3150000
IKEA,300,4550000
Microsoft,300,350000


In [87]:
df.groupby('Company').sum().drop(columns = 'Account Number')

,Licenses,Sale Price
Company,,
ATT,300,1050000
Apple,300,4550000
BOBO,150,2450000
CVS Health,450,490000
Cisco,600,4900000
Exxon Mobile,150,2100000
Google,600,3150000
IKEA,300,4550000
Microsoft,300,350000


The above .groupby() method call gives an identical result. Overall, I think I prefer using this approach, but it's good to know that there are other ways of doing it, and .pivot() is required if you don't want to do any aggregation.

I guess if you've already created the pivot table and you want to do aggregation later, you can do the first method. Otherwise, I'd probably use .groupby()

Another way of doing the same thing as .drop() is to provide the columns you want:

In [88]:
pd.pivot_table(df, index = 'Company', aggfunc = 'sum', values = ['Licenses', 'Sale Price'])

,Licenses,Sale Price
Company,,
ATT,300,1050000
Apple,300,4550000
BOBO,150,2450000
CVS Health,450,490000
Cisco,600,4900000
Exxon Mobile,150,2100000
Google,600,3150000
IKEA,300,4550000
Microsoft,300,350000


Which makes sense if you have a ton of unneeded columns, but in this case, .drop() is way easier

In [89]:
df

,Account Number,Company,Contact,Account Manager,Product,Licenses,Sale Price,Status
0,2123398,Google,Larry Pager,Edward Thorp,Analytics,150,2100000,Presented
1,2123398,Google,Larry Pager,Edward Thorp,Prediction,150,700000,Presented
2,2123398,Google,Larry Pager,Edward Thorp,Tracking,300,350000,Under Review
3,2192650,BOBO,Larry Pager,Edward Thorp,Analytics,150,2450000,Lost
4,420496,IKEA,Elon Tusk,Edward Thorp,Analytics,300,4550000,Won
5,636685,Tesla Inc.,Elon Tusk,Edward Thorp,Analytics,300,2800000,Under Review
6,636685,Tesla Inc.,Elon Tusk,Edward Thorp,Prediction,150,700000,Presented
7,1216870,Microsoft,Will Grates,Edward Thorp,Tracking,300,350000,Under Review
8,2200450,Walmart,Will Grates,Edward Thorp,Analytics,150,2450000,Lost
9,405886,Apple,Cindy Phoner,Claude Shannon,Analytics,300,4550000,Won


In [92]:
pd.pivot_table(df, index = ['Account Manager', 'Contact'], values = 'Sale Price', aggfunc = 'sum')

Sale Price
Account Manager Contact                 
Claude Shannon  Cindy Phoner     7700000
                Emma Gordian    12390000
Edward Thorp    Elon Tusk        8050000
                Larry Pager      5600000
                Will Grates      2800000

The goal of this ^ is to create a multi-index with contacts as the sub-index

Similar to .groupby(), the first argument within index is the outer layer index, the second argument is the inner-layer index

What this means is that at the company I work for in this example only has 2 account managers, and each one of those managers has either 2 or 3 contacts at different companies.

Each contact at the various companies have brought in a certain sum of money through sales, but it may be more useful to ask the total sum brought in by each account manager, depending on the relevant question being asked:

In [93]:
pd.pivot_table(df, index = ['Account Manager'], values = 'Sale Price', aggfunc = 'sum')

,Sale Price
Account Manager,
Claude Shannon,20090000
Edward Thorp,16450000


Now we can see the total sum brought in by each account manager, which are in the tens of millions

In [94]:
pd.pivot_table(df, index = ['Account Manager', 'Contact'], values = 'Sale Price', columns = ['Product'], aggfunc = 'sum')

Product                        Analytics  GPS Positioning  Prediction  \
Account Manager Contact                                                 
Claude Shannon  Cindy Phoner   6650000.0              NaN    700000.0   
                Emma Gordian  11550000.0         350000.0         NaN   
Edward Thorp    Elon Tusk      7350000.0              NaN    700000.0   
                Larry Pager    4550000.0              NaN    700000.0   
                Will Grates    2450000.0              NaN         NaN   

Product                       Tracking  
Account Manager Contact                 
Claude Shannon  Cindy Phoner  350000.0  
                Emma Gordian  490000.0  
Edward Thorp    Elon Tusk          NaN  
                Larry Pager   350000.0  
                Will Grates   350000.0

Using the existing Product column, the total sales price for each product is seperated out by product by contact

Essentially, both the X and Y axes have two-tiers of multi-index: the X has the company's Account Managers broken down by their contacts with other companies, and those contacts have the total Sales Price broken down into all the individual products they deal in

These null values can be automatically filled with 0 if needed:

In [95]:
pd.pivot_table(df, index = ['Account Manager', 'Contact'], values = 'Sale Price', columns = ['Product'], aggfunc = 'sum', fill_value = 0)

Product                       Analytics  GPS Positioning  Prediction  Tracking
Account Manager Contact                                                       
Claude Shannon  Cindy Phoner    6650000                0      700000    350000
                Emma Gordian   11550000           350000           0    490000
Edward Thorp    Elon Tusk       7350000                0      700000         0
                Larry Pager     4550000                0      700000    350000
                Will Grates     2450000                0           0    350000

To make it even more complex:

It's possible to display multiple aggregations, i.e. both sum and mean:

In [96]:
pd.pivot_table(df, index = ['Account Manager', 'Contact'], values = 'Sale Price', columns = ['Product'], aggfunc = [np.sum, np.mean], fill_value = 0)

sum                                      \
Product                      Analytics GPS Positioning Prediction Tracking   
Account Manager Contact                                                      
Claude Shannon  Cindy Phoner   6650000               0     700000   350000   
                Emma Gordian  11550000          350000          0   490000   
Edward Thorp    Elon Tusk      7350000               0     700000        0   
                Larry Pager    4550000               0     700000   350000   
                Will Grates    2450000               0          0   350000   

                                  mean                                      
Product                      Analytics GPS Positioning Prediction Tracking  
Account Manager Contact                                                     
Claude Shannon  Cindy Phoner   3325000               0     700000   350000  
                Emma Gordian   5775000          350000          0   490000  
Edward Thorp    Elon Tusk      3675000               0     700000        0  
                Larry Pager    2275000               0     700000   350000  
                Will Grates    2450000               0          0   350000

This is done using [np.sum, np.mean] etc. The tables are displayed side-by-side. This approach is not wildly easy to read in my opinion and may require some kind of help to be understood clearly. I would honestly prefer to have two different tables for sum and mean unless there's a cleaner way to do this. The one benefit of doing np.sum etc. is that the table is labeled with sum

"You're trying to simplify things, not make it more complecated" - my instructor.

I agree, which is why I would proably not use this approach where sum and mean are in the same dataframe

This might be another good approach for displaying product sales data:

In [97]:
pd.pivot_table(df, index = ['Account Manager', 'Contact', 'Product'], values = 'Sale Price', aggfunc = [np.sum], fill_value = 0)

sum
                                             Sale Price
Account Manager Contact      Product                   
Claude Shannon  Cindy Phoner Analytics          6650000
                             Prediction          700000
                             Tracking            350000
                Emma Gordian Analytics         11550000
                             GPS Positioning     350000
                             Tracking            490000
Edward Thorp    Elon Tusk    Analytics          7350000
                             Prediction          700000
                Larry Pager  Analytics          4550000
                             Prediction          700000
                             Tracking            350000
                Will Grates  Analytics          2450000
                             Tracking            350000

In [98]:
pd.pivot_table(df, index = ['Account Manager', 'Contact', 'Product'], values = 'Sale Price', aggfunc = [np.sum, np.mean], fill_value = 0)

sum       mean
                                             Sale Price Sale Price
Account Manager Contact      Product                              
Claude Shannon  Cindy Phoner Analytics          6650000    3325000
                             Prediction          700000     700000
                             Tracking            350000     350000
                Emma Gordian Analytics         11550000    5775000
                             GPS Positioning     350000     350000
                             Tracking            490000     490000
Edward Thorp    Elon Tusk    Analytics          7350000    3675000
                             Prediction          700000     700000
                Larry Pager  Analytics          4550000    2275000
                             Prediction          700000     700000
                             Tracking            350000     350000
                Will Grates  Analytics          2450000    2450000
                             Tracking            350000     350000

And now ^ showing both sum and mean in the same table is actually pretty clean

In [99]:
pd.pivot_table(df, index = ['Account Manager', 'Contact', 'Product'], values = 'Sale Price', aggfunc = [np.sum, np.mean], fill_value = 0, margins = True)

sum          mean
                                             Sale Price    Sale Price
Account Manager Contact      Product                                 
Claude Shannon  Cindy Phoner Analytics          6650000  3.325000e+06
                             Prediction          700000  7.000000e+05
                             Tracking            350000  3.500000e+05
                Emma Gordian Analytics         11550000  5.775000e+06
                             GPS Positioning     350000  3.500000e+05
                             Tracking            490000  4.900000e+05
Edward Thorp    Elon Tusk    Analytics          7350000  3.675000e+06
                             Prediction          700000  7.000000e+05
                Larry Pager  Analytics          4550000  2.275000e+06
                             Prediction          700000  7.000000e+05
                             Tracking            350000  3.500000e+05
                Will Grates  Analytics          2450000  2.450000e+06
                             Tracking            350000  3.500000e+05
All                                            36540000  2.149412e+06

Using margins = True gives a grand total at the bottom